In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
from astropy.time import Time

In [2]:
ROOT_DIR = r'C:\Users\jgmad\Research\Ibn'
DATA_DIR =  os.path.join(ROOT_DIR, "data")
TXT_DIR = os.path.join(DATA_DIR, "ibn_papers_raw")
OUT_DIR = os.path.join(DATA_DIR,"ibn_papers")
text_files = glob(os.path.join(TXT_DIR, "*"))

In [8]:
for file in text_files:
    filename = os.path.basename(file)
    supernova_name = filename.split("_")[0]

    df = pd.read_csv(
        file,
        comment='#',
        sep=r'\s+',        # split on any run of whitespace
        quotechar='"',     # so "" is parsed as an empty string, not as two quotes
        engine='python',   # the python engine handles regex separators + quoting together
        header=0           # first line is the column names
    )

    # 2) Convert JD → Time objects:
    try:
        t = Time(df['JD'], format='jd')
    except ValueError:
        print(supernova_name)

    # 3) Add the new columns:
    df['dateobs'] = t.iso               # e.g. "2011-03-10 04:18:36.000"
    df['mjd']     = t.mjd               # e.g. 55884.77000000
    df['source'] = 'papers'

    # 4) Rename and reshape:
    df = df.rename(columns={
        'MAG':     'mag',
        'MAG_ERR': 'dmag',
        'FILTER':  'filter',
        'DETECTION': 'magtype'
    })
    df['ant_maglim'] = np.where(df['magtype'] == -1,df['mag'],np.nan)
    df['mag'] = np.where(df['magtype'] == 1,df['mag'],np.nan)

    # 5) Reorder to your target format:
    df = df[['dateobs','mjd','mag','dmag','ant_maglim','source','filter','magtype']]
    df.to_csv(os.path.join(OUT_DIR, f"{supernova_name}.csv"), index=False)
    if len(df[df['magtype'] == -1]) == 0:
        print(f'No nondets found for {supernova_name}')
    print(f'Saved {supernova_name} to {os.path.join(OUT_DIR, f"{supernova_name}.csv")}')

No nondets found for 2011hw
Saved 2011hw to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2011hw.csv
Saved 2014av to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2014av.csv
No nondets found for 2015U
Saved 2015U to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2015U.csv
Saved 2018jmt to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2018jmt.csv
No nondets found for 2019deh
Saved 2019deh to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2019deh.csv
No nondets found for 2019kbj
Saved 2019kbj to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2019kbj.csv
No nondets found for 2019uo
Saved 2019uo to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2019uo.csv
No nondets found for 2019wep
Saved 2019wep to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2019wep.csv
No nondets found for 2020nxt
Saved 2020nxt to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2020nxt.csv
No nondets found for 2021jpk
Saved 2021jpk to C:\Users\jgmad\Research\Ibn\data\ibn_papers\2021jpk.csv
Saved 2023emq to C:\Users\jgmad\Research\Ibn\data\ib

In [30]:
print([os.path.basename(file).split("_")[0] for file in text_files])

['2011hw', '2014av', '2015U', '2018jmt', '2019deh', '2019kbj', '2019uo', '2019wep', '2020nxt', '2021jpk', '2023emq', '2023tsz', '2024acyl', 'iPTF14aki', 'iPTF15akq', 'iPTF15ul', 'ps1-12sk', 'PTF11rfh', 'PTF12ldy']
